To run this notebook, please visit https://colab.research.google.com/drive/1MTzxqOtNFRp4aO4ChuXkh9IZNBDKKpIa

In [2]:
!gdown --id 1nuSfGzNECJAuma3DRjZ0ddqjSZ5Efa9g
!gdown --id 1c4julR3O4xgZexI1EI5UiGc_AoIV-I88
!gdown --id 1xHxYJ_kFXWnuG7nCZVNL8SZ63h8nVvwV
!gdown --id 1oWefOpcG032l7B5GCUbgZjKEUgeZwXqn
!gdown --id 1kOqbLmN5-zvVwdAhYrhiCu2GUPiyg-Qx

Downloading...
From: https://drive.google.com/uc?id=1nuSfGzNECJAuma3DRjZ0ddqjSZ5Efa9g
To: /content/teamsgames.pkl
100% 1.50M/1.50M [00:00<00:00, 100MB/s]
Downloading...
From: https://drive.google.com/uc?id=1c4julR3O4xgZexI1EI5UiGc_AoIV-I88
To: /content/ortgconverted.joblib
209MB [00:01, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xHxYJ_kFXWnuG7nCZVNL8SZ63h8nVvwV
To: /content/drtgconverted.joblib
605MB [00:04, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oWefOpcG032l7B5GCUbgZjKEUgeZwXqn
To: /content/NCAA.py
100% 24.8k/24.8k [00:00<00:00, 44.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kOqbLmN5-zvVwdAhYrhiCu2GUPiyg-Qx
To: /content/model2.torch
100% 498k/498k [00:00<00:00, 67.0MB/s]


In [3]:
!pip install aiohttp async_timeout hummingbird-ml==0.0.6

In [4]:
from NCAA import *
import torch, joblib, itertools, numpy, hummingbird.ml
 
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, hidden4_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden1_size = hidden1_size
        self.hidden2_size = hidden2_size
        self.hidden3_size = hidden3_size
        self.hidden4_size = hidden4_size
        self.relu = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden1_size)
        self.fc2 = torch.nn.Linear(self.hidden1_size, self.hidden2_size)
        self.fc3 = torch.nn.Linear(self.hidden2_size, self.hidden3_size)
        self.fc4 = torch.nn.Linear(self.hidden3_size, self.hidden4_size)
        self.fc5 = torch.nn.Linear(self.hidden4_size, 2)
    
    def forward(self, x):
        hidden1 = self.fc1(x)
        relu1 = self.relu(hidden1)
        hidden2 = self.fc2(relu1)
        relu2 = self.relu(hidden2)
        hidden3 = self.fc3(relu2)
        relu3 = self.relu(hidden3)
        hidden4 = self.fc4(relu3)
        relu4 = self.relu(hidden4)
        hidden5 = self.fc5(relu4)
        output = self.relu(hidden5)
        return output
 
def evaluate_regression(regressor, X, y, samples = 100, std_multiplier = 2):
    preds = [regressor(X) for i in range(samples)]
    preds = torch.stack(preds)
    means = preds.mean(axis=0)
    stds = preds.std(axis=0)
    ci_upper = means + (std_multiplier * stds)
    ci_lower = means - (std_multiplier * stds)
    ic_acc = (ci_lower <= y) * (ci_upper >= y)
    ic_acc = ic_acc.float().mean()
    return ic_acc, (ci_upper >= y).float().mean(), (ci_lower <= y).float().mean()
 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
teams, games = recalldata('teamsgames.pkl')
teams[2934].conference = 'WAC'
teams[140].conference = 'WAC'
teams[2885].conference = 'ASUN'
teams[2870].conference = 'Summit League'
teams[2523].conference = 'Northeast'
teams[41].conference = 'American'
savedata(teams, games, 'teamsgames.pkl')
teams, games = cleannonD1(teams, games)
autobids = {'ASUN': teams[2335], 'Big South': teams[2737], 'Colonial': teams[2275], 'Horizon': teams[94], 'Missouri Valley': teams[71], 'Mountain West': teams[328], 'Northeast': teams[2523], 'Ohio Valley': teams[2057], 'Patriot League': teams[104], 'Southern': teams[2193], 'Summit League': teams[2449], 'West Coast': teams[2250]}
krachrtgs, teamsos = calckrachratings(teams, games)
log100 = math.log(100)
logkrach = {i: math.log(r)/log100-1 for i, r in krachrtgs.items()}
logsos = {i: math.log(r)/log100-1 for i, r in teamsos.items()}
origkrachrtgs = krachrtgs.copy()
origteamsos = teamsos.copy()
ortgs = {teamid: team.offensiverating() for teamid, team in teams.items()}
drtgs = {teamid: team.defensiverating() for teamid, team in teams.items()}
model = Net(12, 200, 200, 200, 200).to(device)
model.load_state_dict(torch.load('model2.torch'))
ortgmodel = joblib.load('ortgconverted.joblib')
drtgmodel = joblib.load('drtgconverted.joblib')
ortgmodel.to('cuda')
drtgmodel.to('cuda')
 
def simulate_game(hometeamid, awayteamid, randomness=True, neutralsite=False):
    hometeam = teams[hometeamid]
    awayteam = teams[awayteamid]
    homeortg = ortgs[hometeamid]
    homedrtg = drtgs[hometeamid]
    awayortg = ortgs[awayteamid]
    awaydrtg = drtgs[awayteamid]
    homeprevgames = len(hometeam.games)
    awayprevgames = len(awayteam.games)
    totalhomepoints = 0
    totalhomepointsagainst = 0
    totalawaypoints = 0
    totalawaypointsagainst = 0
    for game in hometeam.games:
        if hometeamid == game.hometeam.teamid:
            totalhomepoints += game.homescore
            totalhomepointsagainst += game.awayscore
        else:
            totalhomepoints += game.awayscore
            totalhomepointsagainst += game.homescore
    for game in awayteam.games:
        if awayteamid == game.hometeam.teamid:
            totalawaypoints += game.homescore
            totalawaypointsagainst += game.awayscore
        else:
            totalawaypoints += game.awayscore
            totalawaypointsagainst += game.homescore
    avghomepoints = totalhomepoints/homeprevgames
    avghomepointsagainst = totalhomepointsagainst/homeprevgames
    avgawaypoints = totalawaypoints/awayprevgames
    avgawaypointsagainst = totalawaypointsagainst/awayprevgames
    inputs = torch.FloatTensor([logkrach[hometeamid]-logkrach[awayteamid], logsos[hometeamid], logsos[awayteamid], homeortg-100, homedrtg-100, avghomepoints, avghomepointsagainst, awayortg-100, awaydrtg-100, avgawaypoints, avgawaypointsagainst, neutralsite]).to(device)
    a = model(inputs)
    homescore = a[0].item()
    awayscore = a[1].item()
    if randomness:
        if neutralsite:
            deltas = numpy.random.multivariate_normal(neutralmean, neutralcov)
        else:
            deltas = numpy.random.multivariate_normal(regularmean, regularcov)
        rhomescore = int(round(homescore+deltas[0]))
        rawayscore = int(round(awayscore+deltas[1]))
        while rhomescore == rawayscore:
            if neutralsite:
                deltas = numpy.random.multivariate_normal(neutralmean, neutralcov)
            else:
                deltas = numpy.random.multivariate_normal(regularmean, regularcov)
            rhomescore = int(round(homescore+deltas[0]))
            rawayscore = int(round(awayscore+deltas[1]))
    else:
        rhomescore = int(round(homescore))
        rawayscore = int(round(awayscore))
    return rhomescore, rawayscore
 
l1regular = []
l2regular = []
l1neutral = []
l2neutral = []
for game in games.values():
    if not game.neutralsite:
        a = simulate_game(game.hometeam.teamid, game.awayteam.teamid, randomness=False, neutralsite=False)
        l1regular.append(game.homescore-a[0])
        l2regular.append(game.awayscore-a[1])
    else:
        a = simulate_game(game.hometeam.teamid, game.awayteam.teamid, randomness=False, neutralsite=True)
        l1neutral.append(game.homescore-a[0])
        l2neutral.append(game.awayscore-a[1])
 
regularmean = numpy.array([numpy.mean(l1regular), numpy.mean(l2regular)])
regularcov = numpy.cov([l1regular, l2regular])
neutralmean = numpy.array([numpy.mean(l1neutral), numpy.mean(l2neutral)])
neutralcov = numpy.cov([l1neutral, l2neutral])
 
def run_pairing(gameid, hometeamid, awayteamid, neutralsite=False):
    global hometeam, awayteam, teams, games
    hometeam = teams[hometeamid]
    awayteam = teams[awayteamid]
    homescore, awayscore = simulate_game(hometeamid, awayteamid, neutralsite=neutralsite)
    game = Game(gameid, hometeam=hometeam, homescore=homescore, awayteam=awayteam, awayscore=awayscore, neutralsite=neutralsite)
    newortghome = ortgmodel.predict([[1-0.5*neutralsite, int(game.winner() is hometeam), game.homescore, game.awayscore, len(hometeam.games), ortgs[hometeamid], drtgs[hometeamid], krachrtgs[hometeamid], hometeam.D1winpct(), teamsos[hometeamid]]])
    newdrtghome = drtgmodel.predict([[1-0.5*neutralsite, int(game.winner() is hometeam), game.homescore, game.awayscore, len(hometeam.games), ortgs[hometeamid], drtgs[hometeamid], krachrtgs[hometeamid], hometeam.D1winpct(), teamsos[hometeamid]]])
    newortgaway = ortgmodel.predict([[0.5*neutralsite, int(game.winner() is awayteam), game.homescore, game.awayscore, len(awayteam.games), ortgs[awayteamid], drtgs[awayteamid], krachrtgs[awayteamid], awayteam.D1winpct(), teamsos[awayteamid]]])
    newdrtgaway = drtgmodel.predict([[0.5*neutralsite, int(game.winner() is awayteam), game.homescore, game.awayscore, len(awayteam.games), ortgs[awayteamid], drtgs[awayteamid], krachrtgs[awayteamid], awayteam.D1winpct(), teamsos[awayteamid]]])
    ortgs[hometeamid] = newortghome.item()
    drtgs[hometeamid] = newdrtghome.item()
    ortgs[awayteamid] = newortgaway.item()
    drtgs[awayteamid] = newdrtgaway.item()
    games[gameid] = game
    hometeam.addgame(game)
    awayteam.addgame(game)
    return game
 
def reset():
    global teams, games, autobids, krachrtgs, teamsos, logkrach, logsos, ortgs, drtgs
    teams, games = recalldata('teamsgames.pkl')
    teams, games = cleannonD1(teams, games)
    autobids = {'ASUN': teams[2335], 'Big South': teams[2737], 'Colonial': teams[2275], 'Horizon': teams[94], 'Missouri Valley': teams[71], 'Mountain West': teams[328], 'Northeast': teams[2523], 'Ohio Valley': teams[2057], 'Patriot League': teams[104], 'Southern': teams[2193], 'Summit League': teams[2449], 'West Coast': teams[2250]}
    krachrtgs = origkrachrtgs.copy()
    teamsos = origteamsos.copy()
    logkrach = {i: math.log(r)/log100-1 for i, r in krachrtgs.items()}
    logsos = {i: math.log(r)/log100-1 for i, r in teamsos.items()}
    ortgs = {teamid: team.offensiverating() for teamid, team in teams.items()}
    drtgs = {teamid: team.defensiverating() for teamid, team in teams.items()}
 
def simulate_america_east():
    final = run_pairing(1, 42, 261, neutralsite=False)
    autobids['America East'] = final.winner()
 
def simulate_aac():
    round1_1 = run_pairing(2, 2116, 58, neutralsite=True)
    round1_2 = run_pairing(3, 41, 2655, neutralsite=True)
    round1_3 = run_pairing(4, 2567, 218, neutralsite=True)
    round1_4 = run_pairing(5, 235, 151, neutralsite=True)
    quarter_1 = run_pairing(6, 2132, round1_1.winner().teamid, neutralsite=True)
    quarter_2 = run_pairing(7, 2724, round1_2.winner().teamid, neutralsite=True)
    quarter_3 = run_pairing(8, 248, round1_3.winner().teamid, neutralsite=True)
    quarter_4 = run_pairing(9, 202, round1_4.winner().teamid, neutralsite=True)
    semi_1 = run_pairing(10, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(11, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(12, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['American'] = final.winner()
 
def simulate_atlantic_10():
    round2_1 = run_pairing(13, 113, 2670, neutralsite=True)
    round2_2 = run_pairing(14, 179, 2244, neutralsite=True)
    round2_3 = run_pairing(15, 2166, 2325, neutralsite=True)
    round2_4 = run_pairing(16, 2184, 2230, neutralsite=True)
    quarter_1 = run_pairing(17, 2168, round2_1.winner().teamid, neutralsite=True)
    quarter_2 = run_pairing(18, 139, round2_2.winner().teamid, neutralsite=True)
    quarter_3 = run_pairing(19, 257, round2_3.winner().teamid, neutralsite=True)
    quarter_4 = run_pairing(20, 227, round2_4.winner().teamid, neutralsite=True)
    semi_1 = run_pairing(21, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(22, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(23, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Atlantic 10'] = final.winner()
 
def simulate_acc():
    quarter_1 = run_pairing(24, 52, 228, neutralsite=True)
    quarter_2 = run_pairing(25, 150, 152, neutralsite=True)
    quarter_3 = run_pairing(26, 258, 87, neutralsite=True)
    quarter_4 = run_pairing(27, 97, 183, neutralsite=True)
    semi_1 = run_pairing(28, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(29, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(30, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['ACC'] = final.winner()
 
def simulate_big_12():
    quarter_1 = run_pairing(31, 2305, 197, neutralsite=True)
    quarter_2 = run_pairing(32, 251, 2641, neutralsite=True)
    quarter_3 = run_pairing(33, 239, 2306, neutralsite=True)
    quarter_4 = run_pairing(34, 201, 277, neutralsite=True)
    semi_1 = run_pairing(35, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(36, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(37, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Big 12'] = final.winner()
 
def simulate_big_east():
    quarter_1 = run_pairing(38, 156, 2599, neutralsite=True)
    quarter_2 = run_pairing(39, 2507, 2086, neutralsite=True)
    quarter_3 = run_pairing(40, 222, 305, neutralsite=True)
    quarter_4 = run_pairing(41, 2550, 269, neutralsite=True)
    semi_1 = run_pairing(42, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(43, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(44, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Big East'] = final.winner()
 
def simulate_big_sky():
    quarter_1 = run_pairing(45, 331, 16, neutralsite=True)
    quarter_2 = run_pairing(46, 2502, 147, neutralsite=True)
    quarter_3 = run_pairing(47, 2458, 253, neutralsite=True)
    quarter_4 = run_pairing(48, 149, 304, neutralsite=True)
    semi_1 = run_pairing(49, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(50, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(51, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Big Sky'] = final.winner()
 
def simulate_big_10():
    round2_1 = run_pairing(52, 164, 130, neutralsite=True)
    round2_2 = run_pairing(53, 2294, 135, neutralsite=True)
    round2_3 = run_pairing(54, 194, 2509, neutralsite=True)
    round2_4 = run_pairing(55, 213, 84, neutralsite=True)
    quarter_1 = run_pairing(56, 275, round2_1.winner().teamid, neutralsite=True)
    quarter_2 = run_pairing(57, 356, round2_2.winner().teamid, neutralsite=True)
    quarter_3 = run_pairing(58, 127, round2_3.winner().teamid, neutralsite=True)
    quarter_4 = run_pairing(59, 120, round2_4.winner().teamid, neutralsite=True)
    semi_1 = run_pairing(60, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(61, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(62, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Big Ten'] = final.winner()
 
def simulate_big_west():
    preseed = {300: 1, 2463: 2, 2540: 3, 62: 4, 302: 5, 27: 6, 2239: 7, 299: 8}
    quarter_1 = run_pairing(63, 300, 299, neutralsite=True)
    quarter_2 = run_pairing(64, 62, 302, neutralsite=True)
    quarter_3 = run_pairing(65, 2463, 2239, neutralsite=True)
    quarter_4 = run_pairing(66, 2540, 27, neutralsite=True)
    quarterwinners = sorted([quarter_1.winner(), quarter_2.winner(), quarter_3.winner(), quarter_4.winner()], key=lambda i: preseed[i.teamid])
    semi_1 = run_pairing(67, quarterwinners[0].teamid, quarterwinners[3].teamid, neutralsite=True)
    semi_2 = run_pairing(68, quarterwinners[1].teamid, quarterwinners[2].teamid, neutralsite=True)
    final = run_pairing(69, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Big West'] = final.winner()
 
def simulate_cusa():
    quarter_1 = run_pairing(70, 249, 2226, neutralsite=True)
    quarter_2 = run_pairing(71, 2429, 2229, neutralsite=True)
    quarter_3 = run_pairing(72, 98, 5, neutralsite=True)
    quarter_4 = run_pairing(73, 2348, 276, neutralsite=True)
    semi_1 = run_pairing(74, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(75, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(76, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Conference USA'] = final.winner()
 
def simulate_ivy_league():
    semi_1 = run_pairing(77, 43, 219, neutralsite=True)
    semi_2 = run_pairing(78, 108, 163, neutralsite=False)
    final = run_pairing(79, semi_2.winner().teamid, semi_1.winner().teamid, neutralsite=not (semi_2.winner().teamid == 108))
    autobids['Ivy'] = final.winner()
 
def simulate_maac():
    quarter_2 = run_pairing(80, 2405, 2514, neutralsite=True)
    quarter_4 = run_pairing(81, 2520, 315, neutralsite=True)
    semi_1 = run_pairing(82, 2561, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(83, 2612, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(84, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['MAAC'] = final.winner()
 
def simulate_mid_american():
    quarter_1 = run_pairing(85, 2006, 195, neutralsite=True)
    quarter_2 = run_pairing(86, 2459, 193, neutralsite=True)
    quarter_3 = run_pairing(87, 189, 2649, neutralsite=True)
    quarter_4 = run_pairing(88, 2050, 2309, neutralsite=True)
    semi_1 = run_pairing(89, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(90, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(91, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Mid-American'] = final.winner()
 
def simulate_meac():
    quarter_2 = run_pairing(92, 2065, 2415, neutralsite=True)
    quarter_4 = run_pairing(93, 2450, 2154, neutralsite=True)
    semi_1 = run_pairing(94, 2428, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(95, 2448, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(96, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['MEAC'] = final.winner()
 
def simulate_pac_12():
    quarter_1 = run_pairing(97, 2483, 204, neutralsite=True)
    quarter_2 = run_pairing(98, 30, 12, neutralsite=True)
    quarter_3 = run_pairing(99, 26, 25, neutralsite=True)
    quarter_4 = run_pairing(100, 9, 265, neutralsite=True)
    semi_1 = run_pairing(101, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(102, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(103, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Pac-12'] = final.winner()
 
def simulate_sec():
    round2_1 = run_pairing(104, 2633, 333, neutralsite=True)
    round2_2 = run_pairing(105, 57, 61, neutralsite=True)
    round2_3 = run_pairing(106, 245, 142, neutralsite=True)
    round2_4 = run_pairing(107, 2579, 8, neutralsite=True)
    quarter_1 = run_pairing(108, 96, round2_1.winner().teamid, neutralsite=True)
    quarter_2 = run_pairing(109, 344, round2_2.winner().teamid, neutralsite=True)
    quarter_3 = run_pairing(110, 2, round2_3.winner().teamid, neutralsite=True)
    quarter_4 = run_pairing(111, 99, round2_4.winner().teamid, neutralsite=True)
    semi_1 = run_pairing(112, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(113, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(114, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['SEC'] = final.winner()
 
def simulate_southland():
    round2_1 = run_pairing(115, 2534, 2466, neutralsite=True)
    round2_2 = run_pairing(116, 2447, 2320, neutralsite=True)
    semi_1 = run_pairing(117, 2617, round2_1.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(118, 2000, round2_2.winner().teamid, neutralsite=True)
    final = run_pairing(119, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['Southland'] = final.winner()
 
def simulate_swac():
    semi_1 = run_pairing(120, 2504, 2296, neutralsite=True)
    semi_2 = run_pairing(121, 2582, 2640, neutralsite=True)
    final = run_pairing(122, semi_2.winner().teamid, semi_1.winner().teamid, neutralsite=True)
    autobids['SWAC'] = final.winner()
 
def simulate_sun_belt():
    semi_1 = run_pairing(123, 2031, 290, neutralsite=True)
    semi_2 = run_pairing(124, 6, 326, neutralsite=True)
    final = run_pairing(125, semi_2.winner().teamid, semi_1.winner().teamid, neutralsite=True)
    autobids['Sun Belt'] = final.winner()
 
def simulate_wac():
    quarter_1 = run_pairing(126, 166, 2130, neutralsite=True)
    quarter_2 = run_pairing(127, 2253, 140, neutralsite=True)
    quarter_3 = run_pairing(128, 292, 2934, neutralsite=True)
    quarter_4 = run_pairing(129, 2547, 3084, neutralsite=True)
    semi_1 = run_pairing(130, quarter_1.winner().teamid, quarter_2.winner().teamid, neutralsite=True)
    semi_2 = run_pairing(131, quarter_3.winner().teamid, quarter_4.winner().teamid, neutralsite=True)
    final = run_pairing(132, semi_1.winner().teamid, semi_2.winner().teamid, neutralsite=True)
    autobids['WAC'] = final.winner()
 
def simulate_conference_tourneys():
    simulate_america_east()
    simulate_aac()
    simulate_atlantic_10()
    simulate_acc()
    simulate_big_12()
    simulate_big_east()
    simulate_big_sky()
    simulate_big_10()
    simulate_big_west()
    simulate_cusa()
    simulate_ivy_league()
    simulate_maac()
    simulate_mid_american()
    simulate_meac()
    simulate_pac_12()
    simulate_sec()
    simulate_southland()
    simulate_swac()
    simulate_sun_belt()
    simulate_wac()
 
def evaluate_seeding_badness(regions):
    badness = 0
    for j in range(1, 5):
        for s in range(1, 9):
            if regions[j][s].conference == regions[j][17-s].conference:
                badness += 4
        for s in range(1, 5):
            badness += 2*(4-len(set(regions[j][u].conference for u in [s, 17-s, 9-s, 8+s])))
        for s in range(1, 3):
            badness += 8-len(set(regions[j][u].conference for u in [s, 17-s, 9-s, 8+s, 5-s, 12+s, 4+s, 13-s]))
    return badness
 
possiblereg = list(itertools.permutations([1, 2, 3, 4]))
def make_assignment(l, trueseeds):
    regions = dict()
    for j in range(1, 5):
        regions[j] = dict()
    for i in range(2, 17):
        for j, k in zip(l[i-2], range(4)):
            regions[j][i] = trueseeds[i][k]
    return regions
 
def autoseed_NCAA():
    tourneyteams = set(autobids.values())
    allteams = sorted(teams.values(), key=lambda t: krachrtgs[t.teamid], reverse=True)
    while len(tourneyteams) < 68:
        tourneyteams.add(allteams.pop(0))
    seedlist = dict()
    autobidseeds = dict()
    atlargeseeds = dict()
    for i, j in zip(range(len(tourneyteams)), sorted(tourneyteams, key=lambda t: krachrtgs[t.teamid], reverse=True)):
        seedlist[i+1] = j
    i = 1
    j = 1
    for k, t in sorted(seedlist.items(), key=lambda i: i[0]):
        if t in autobids.values():
            autobidseeds[i] = t
            i += 1
        else:
            atlargeseeds[j] = t
            j += 1
    seedlist = dict()
    toreseed = {atlargeseeds[i] for i in range(1,33)}.union({autobidseeds[i] for i in range(1,29)})
    for i, j in zip(range(len(toreseed)), sorted(toreseed, key=lambda t: krachrtgs[t.teamid], reverse=True)):
        seedlist[i+1] = j
    firstfour_1 = run_pairing(133, atlargeseeds[33].teamid, atlargeseeds[34].teamid, neutralsite=True)
    firstfour_2 = run_pairing(134, atlargeseeds[35].teamid, atlargeseeds[36].teamid, neutralsite=True)
    firstfour_3 = run_pairing(135, autobidseeds[29].teamid, autobidseeds[30].teamid, neutralsite=True)
    firstfour_4 = run_pairing(136, autobidseeds[31].teamid, autobidseeds[32].teamid, neutralsite=True)
    trueseeds = dict()
    i = 1
    for s in range(1, 17):
        trueseeds[s] = set()
        if s == 11:
            trueseeds[s].update({firstfour_1.winner(), firstfour_2.winner()})
        if s == 16:
            trueseeds[s].update({firstfour_3.winner(), firstfour_4.winner()})
        while len(trueseeds[s]) < 4:
            trueseeds[s].add(seedlist[i])
            i += 1
        trueseeds[s] = list(trueseeds[s])
    regions = dict()
    for j, t in zip(range(1, 5), trueseeds[1]):
        regions[j] = dict()
        regions[j][1] = t
    regionposs = []
    for i in range(10000):
        testregions = dict()
        for j, d in regions.items():
            testregions[j] = dict(d)
        regionsrest = make_assignment(list(map(lambda i: possiblereg[i], numpy.random.randint(24, size=15))), trueseeds)
        for j in range(1, 5):
            testregions[j].update(regionsrest[j])
        regionposs.append(testregions)
    regions = min(regionposs, key=lambda regions: evaluate_seeding_badness(regions))
    return regions
 
def run_all():
    reset()
    simulate_conference_tourneys()
    strrep = 'Conference Tournament Winners:\n'
    for i in sorted(autobids.items(), key=lambda i: i[0]):
        strrep += f'{i[0]}: {i[1].fullname}\n'
    krachrtgs, teamsos = calckrachratings(teams, games)
    logkrach = {i: math.log(r)/log100-1 for i, r in krachrtgs.items()}
    logsos = {i: math.log(r)/log100-1 for i, r in teamsos.items()}
    regions = autoseed_NCAA()
    strrep += '\n'
    for j in range(1, 5):
        strrep += f'Region {j}\n'
        for k in range(1, 17):
            strrep += f'{k}: {regions[j][k].fullname}\n'
        strrep += '\n'
    round1 = dict()
    round2 = dict()
    round3 = dict()
    quarter = dict()
    semi = dict()
    gamenum = 137
    for j in range(1, 5):
        strrep += f'Region {j} results\n'
        strrep += f'Round 1:\n'
        round1[j] = dict()
        for s in range(1, 9):
            team1 = regions[j][s]
            team2 = regions[j][17-s]
            game = run_pairing(gamenum, team1.teamid, team2.teamid, neutralsite=True)
            round1[j][s] = game
            strrep += f'#{s} {team1.fullname} vs. #{17-s} {team2.fullname} ends with score of {game.homescore}-{game.awayscore}\n'
            gamenum += 1
        strrep += f'\nRound 2:\n'
        round2[j] = dict()
        for s in range(1, 5):
            team1 = round1[j][s].winner()
            team2 = round1[j][9-s].winner()
            game = run_pairing(gamenum, team1.teamid, team2.teamid, neutralsite=True)
            round2[j][s] = game
            strrep += f'{team1.fullname} vs. {team2.fullname} ends with score of {game.homescore}-{game.awayscore}\n'
            gamenum += 1
        strrep += f'\nRound 3:\n'
        round3[j] = dict()
        for s in range(1, 3):
            team1 = round2[j][s].winner()
            team2 = round2[j][5-s].winner()
            game = run_pairing(gamenum, team1.teamid, team2.teamid, neutralsite=True)
            round3[j][s] = game
            strrep += f'{team1.fullname} vs. {team2.fullname} ends with score of {game.homescore}-{game.awayscore}\n'
            gamenum += 1
        strrep += '\nRegional Finals:\n'
        team1 = round3[j][1].winner()
        team2 = round3[j][2].winner()
        game = run_pairing(gamenum, team1.teamid, team2.teamid, neutralsite=True)
        quarter[j] = game
        strrep += f'{team1.fullname} vs. {team2.fullname} ends with score of {game.homescore}-{game.awayscore}\n\n'
        gamenum += 1
    strrep += '\nFinal Four:\n'
    for j in range(1, 3):
        team1 = quarter[j].winner()
        team2 = quarter[5-j].winner()
        game = run_pairing(gamenum, team1.teamid, team2.teamid, neutralsite=True)
        semi[j] = game
        strrep += f'{team1.fullname} vs. {team2.fullname} ends with score of {game.homescore}-{game.awayscore}\n'
        gamenum += 1
    strrep += '\nChampionship Game:\n'
    team1 = semi[1].winner()
    team2 = semi[2].winner()
    game = run_pairing(gamenum, team1.teamid, team2.teamid, neutralsite=True)
    final = game
    strrep += f'{team1.fullname} vs. {team2.fullname} ends with score of {game.homescore}-{game.awayscore}'
    return strrep, round1, round2, round3, quarter, semi, final

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from collections import Counter
import os
 
os.system('cp /content/drive/MyDrive/runs.txt .')
 
c = Counter()
for i in range(1500):
    strrep, round1, round2, round3, quarter, semi, final = run_all()
    with open(f'runs.txt', 'a') as f:
        f.write('\n\n\n')
        f.write(strrep)
        f.write('\n\n\n')
    winner = final.winner().fullname
    print(f'Winner is {winner}\n')
    c[winner] += 1
    if (i+1) % 25 == 0:
        os.system('cp runs.txt /content/drive/MyDrive')

Winner is Kansas Jayhawks

Winner is Kansas Jayhawks

Winner is Duke Blue Devils

Winner is Michigan State Spartans

Winner is Kansas Jayhawks

Winner is Marquette Golden Eagles

Winner is Kansas Jayhawks

Winner is Louisville Cardinals

Winner is Gonzaga Bulldogs

Winner is San Diego State Aztecs

Winner is San Diego State Aztecs

Winner is Iowa Hawkeyes

Winner is Creighton Bluejays

Winner is San Diego State Aztecs

Winner is Kansas Jayhawks

Winner is Louisville Cardinals

Winner is Kansas Jayhawks

Winner is Baylor Bears

Winner is San Diego State Aztecs

Winner is Creighton Bluejays

Winner is San Diego State Aztecs

Winner is Kansas Jayhawks

Winner is Dayton Flyers

Winner is BYU Cougars

Winner is Kentucky Wildcats

Winner is Kansas Jayhawks

Winner is Kansas Jayhawks

Winner is Ohio State Buckeyes

Winner is Louisville Cardinals

Winner is Duke Blue Devils

Winner is Kansas Jayhawks

Winner is Kansas Jayhawks

Winner is Kansas Jayhawks

Winner is Oregon Ducks

Winner is Gonza